In [1]:
import pandas as pd

In [ ]:
# Yarışma kapsamında ya bu kısım çıkarılacak ya da verilen veri kümesi TDDI2022 klasörüne koyulacaktır.
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
%cd TDDI2022/
%ls

In [3]:
# Kategori bulmak için çalışacak fonksiyonumuz
def kategoriBul(icerik):
  # Gelen içeriği kategori bulmak için temizleyeceğiz.
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
  icerik = icerik.replace('.','')
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')

  # data_text alanındaki kayıdın ilk parçasında kuralımıza göre kontrol yapacağız!
  if not(icerik.find('yayimlanir')!=-1) and (icerik.find('kanunno')!=-1)and((not(icerik.find('tesrinievvel1336')!=-1))and(not(icerik.find('13361920')!=-1))):
    return 'Kanun'
  elif (icerik.find('bbkno')!=-1) or (icerik.find('bkkno')!=-1):
    return 'Tüzük'
  elif (not(icerik.find('kurulustarihi')!=-1)) and (not(icerik.find('yayimlanir')!=-1)) and icerik.find('khk')!=-1:
    return 'Kanun Hükmünde Kararname'
  elif not(icerik.find('tcresmigazete')!=-1) and not(icerik.find('tesrinievvel')!=-1) and not(icerik.find('yayimlanir')!=-1) and not(icerik.find('teblig')!=-1) and not(icerik.find('yonetmeli')!=-1) and not(icerik.find('genelyazi')!=-1)and(not(icerik.find('icgenelge')!=-1)and((not(icerik.find('tesrinievvel1336')!=-1))and(not(icerik.find('13361920')!=-1))))and(icerik.find('kanuntasarisi')!=-1 or icerik.find('yasamayili')!=-1 or icerik.find('tbmm')!=-1 or icerik.find('komisyonraporu')!=-1 or icerik.find('ssayisi')!=-1 or icerik.find('sirano')!=-1 or icerik.find('ilave')!=-1 or icerik.find('mazbata')!=-1 or icerik.find('hukumet')!=-1 or icerik.find('millet')!=-1):
    return 'Komisyon Raporu'
  elif  (icerik.find('genelge')!=-1 or icerik.find('icgene')!=-1 or icerik.find('tcsos')!=-1 or icerik.find('genelyazi')!=-1) and not(icerik[:icerik.find('genelge')].find('7ekim1920')!=-1):
    return 'Genelge'
  elif (not(icerik[:50].find('gelir')!=-1)) and (not(icerik.find('tbmm')!=-1)) and (((icerik.find('basbakanlik')!=-1) or (icerik.find('tesrinievvel1336')!=-1) or (icerik.find('1336-1920')!=-1)) and ((not(icerik.find('yonetmelik')!=-1))or(not(icerik.find('bbkno')!=-1))or(not(icerik.find('bkkno')!=-1)))):
    return 'Resmi Gazete'
  elif icerik.find('cumhurbaskanligikararnamesi')!=-1:
    return 'Cumhurbaşkanlığı Kararnamesi'
  elif not(icerik.find('yonetmeli')!=-1) and not(icerik.find('7ekim1920')!=-1) and (icerik.find('gelir') != -1 or icerik.find('beyan') != -1 or icerik.find('siparis') != -1)and(not(icerik[:icerik.find('gelir')].find('tebl')!=-1))and(not(icerik[:icerik.find('gelir')].find('1336')!=-1)) :
    return 'Özelge'
  elif (icerik.find('yonetmeli')!=-1) and (not(icerik.find('bbkno')!=-1)) and (not(icerik.find('bkkno')!=-1)) and (not(icerik.find('sayiliresmigazete')!=-1)) and (not(icerik.find('kurulustarihi')!=-1)) and (not(icerik.find('tebl')!=-1)):
    return 'Yönetmelik'
  elif icerik.find('tebl')!=-1 and not(icerik.find('13361920')!=-1) and not(icerik.find('yayimlanir')!=-1):
    return 'Tebliğ'
  else:
    return 'Resmi Gazete'

In [ ]:
# YARIŞMA ESNASINDA VERİ KÜMESİ BOŞ OLACAĞI İÇİN BU HÜCRE ÇALIŞTIRILMAYACAKTIR!

from sklearn.metrics import confusion_matrix
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv') # Yarışma öncesi analiz için kullanılacak veri kümesi
df.info(verbose=True)

# Veri iskeletimiz içinde toplam kaç satir / kayıt ve sütun / alan olduğunu satirToplami ve sutunToplami değişkenlerinde saklıyoruz.
satirToplami, sutunToplami = df.shape

# Tahmin edilen ve olmasi gereken kategorileri bir yerde tutalım!
tahminEdilenKategoriler = []
olmasiGerekenKategoriler = []

dogruSay = 0
# Tüm kategorilerde kontrolümüzü yapıyoruz
for satir in range(satirToplami):
  # kaydi once butunuyle aliyoruz.
  icerik = df['data_text'][satir]
  # Verimizi daha önce izah ettiğimiz gibi iki parcaya ayırıp ilk parçayı alıyoruz!
  b1, b2 = icerik.find('sayı'), icerik.find('madde1')
  b1 = max(b1,200)
  bolmeYeri = b2 if b2!=-1 else b1 if b1!=-1 else 300
  arastirilacakParca = df['data_text'][satir][:bolmeYeri]
  tahminEdilenKategoriler.append(kategoriBul(arastirilacakParca))
  olmasiGerekenKategoriler.append(df['kategori'][satir])
  if tahminEdilenKategoriler[satir]==olmasiGerekenKategoriler[satir]:
    dogruSay += 1
#  else:
#   print(satir,tahminEdilenKategoriler[satir],olmasiGerekenKategoriler[satir])

print('Kategori bulma fonksiyonunun mevcut veri kümesindeki performansı: ',dogruSay/4142)
print('Kategorisini doğru tahmin ettiği belge sayısı: ',dogruSay)
print('Kategorisini hatalı tahmin ettiği belge sayısı: ',4142-dogruSay)

In [ ]:
# YARIŞMA ESNASINDA VERİ KÜMESİ BOŞ OLACAĞI İÇİN BU HÜCRE ÇALIŞTIRILMAYACAKTIR!

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
%matplotlib inline

karmasiklikMatrisi = confusion_matrix(olmasiGerekenKategoriler, tahminEdilenKategoriler, labels=list(set(olmasiGerekenKategoriler)))
kM_gorsel = ConfusionMatrixDisplay(confusion_matrix = karmasiklikMatrisi, display_labels = list(set(olmasiGerekenKategoriler)))

kM_gorsel.plot()
plt.xlabel("Tahmin Edilen")
plt.ylabel("Gerçek Durum")
plt.title("Karmaşıklık Matrisi", fontdict = {'family':'serif','color':'Darkblue','size':20})
plt.xticks([]) # Grafiğin altı karışmasın diye etiketleri kaldırdım.
plt.show()

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Doğruluk (Accuracy):',accuracy_score(olmasiGerekenKategoriler, tahminEdilenKategoriler))
print('Keskinlik (Precision):',precision_score(olmasiGerekenKategoriler, tahminEdilenKategoriler, average='macro'))
print('Hassasiyet (Sensitivity) / Duyarlılık (Recall):',recall_score(olmasiGerekenKategoriler, tahminEdilenKategoriler, average='macro'))
#print('Özgüllük (Specificity):',accuracy_score(olmasiGerekenKategoriler, tahminEdilenKategoriler))
print('F1-Puanı (F1 Score):',f1_score(olmasiGerekenKategoriler, tahminEdilenKategoriler, average='macro'))

In [4]:
veriIskeleti = pd.read_csv('kanunum-nlp-doc-analysis-eval-dataset.csv') # Yarışma kapsamında doldurulacak veri kümesi
veriIskeleti.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4126 entries, 0 to 4125
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4126 non-null   int64  
 1   kategori        0 non-null      float64
 2   baslik          0 non-null      float64
 3   rega_no         0 non-null      float64
 4   mukerrer_no     4126 non-null   int64  
 5   rega_tarihi     0 non-null      float64
 6   kurum           0 non-null      float64
 7   mevzuat_no      0 non-null      float64
 8   belge_sayi      0 non-null      float64
 9   mevzuat_tarihi  0 non-null      float64
 10  donem           0 non-null      float64
 11  sira_no         0 non-null      float64
 12  madde_sayisi    4126 non-null   int64  
 13  data_text       4126 non-null   object 
 14  url             0 non-null      float64
 15  kanunum_url     0 non-null      float64
dtypes: float64(12), int64(3), object(1)
memory usage: 515.9+ KB


In [5]:
def madde_sayisiBul(icerik):
  # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
  maddeSayisi = 0
  # YÖNTEMİMİZİ GELİŞTİRİYORUZ
  geciciMaddeSayisi = 0 # Geçici Madde 1 gibi verilen ek madde
  ekMaddeSayisi = 0 # Madde içinde 10/a gibi verilen ek madde
  ekMaddeSayisi2 = 0 # Ek Madde 1 gibi verilen ek madde
  # İlk maddenin başladığı yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
  # data_text alanında yer alan içeriği madde sayısı bulmak için temizleyeceğiz.
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
#  icerik = icerik.replace('-','') Madde sayısında tire önemli!
  icerik = icerik.replace(',','')
  icerik = icerik.replace('.','')
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
#  icerik = icerik.replace('─','') Madde sayısında tire önemli!
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')

  ilkMaddeninBasladigiYer = icerik.find('madde1')
  if ilkMaddeninBasladigiYer==-1:
    return 1
  verininBuyuklugu = len(icerik)

  # Aramaya baslayacağımız yeri belirliyoruz.
  aramaYapilacakYerinBasi = ilkMaddeninBasladigiYer

  while (aramaYapilacakYerinBasi<verininBuyuklugu):
    bulunanSayi = ''
    alinanKesit = icerik[aramaYapilacakYerinBasi:]
    maddeninBaslayabilecegiYer = alinanKesit.find('madde')
    tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–') if alinanKesit[maddeninBaslayabilecegiYer:].find('–')!=-1 else alinanKesit[maddeninBaslayabilecegiYer:].find('-')
    if abs(tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=9:
      maddeninBaslayabilecegiYer+=5
      while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
        maddeninBaslayabilecegiYer+=1
        bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
        if bulunanSayiyiOlusturacakParca in '0123456789':
          bulunanSayi+=bulunanSayiyiOlusturacakParca
        else:
              break
      if bulunanSayi!='':
        if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
          maddeSayisi += int(bulunanSayi)-maddeSayisi
        elif int(bulunanSayi)<geciciMaddeSayisi+2:
          geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
        elif int(bulunanSayi)<ekMaddeSayisi2+2:
          ekMaddeSayisi2 += int(bulunanSayi)-ekMaddeSayisi2
    else:
      ekMaddeninYeri = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('/')
      if (ekMaddeninYeri<=(tireninBaslayabilecegiYer-2)) and ((tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=12):
        ekMaddeSayisi += 1
    aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
  maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+ekMaddeSayisi2)
  return maddeSayisi

In [6]:
# rega_no ve rega_tarihi bulan fonksiyon

def ayinAgirliginaBakipTahminYap(gelenVeri):
  gelenVeri=gelenVeri.replace('ṡ','s')
  gelenVeri=gelenVeri.replace('ṙ','r')
  gelenVeri=gelenVeri.replace('ṁ','m')
  gelenVeri=gelenVeri.replace('ṅ','n')
  gelenVeri=gelenVeri.replace('ı','i')
  gelenVeri=gelenVeri.replace('ş','s')
  gelenVeri=gelenVeri.replace('ğ','g')
  gelenVeri=gelenVeri.replace('ü','u')
  gelenVeri=gelenVeri.replace('i̇','i')
  gelenVeri=gelenVeri.replace('î','i')
  hesapliListe = [414, 543, 436, 537, 547, 749, 674, 774, 555, 422, 533, 628]
  agirlik = 0
  for g in gelenVeri:
    agirlik += ord(g)
  farklar = []
  for s in hesapliListe:
    farklar.append(abs(s-agirlik))
  return farklar.index(min(farklar))+1


def regaNo(icerik):
  # data_text alanında yer alan içeriği regaNo bulmak için temizleyeceğiz.
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
  icerik = icerik.replace('.','')
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  
  icerik.strip()
  if icerik=='':
    return 0

  sayininOlduguYer = icerik.find('sayi')
  if sayininOlduguYer==-1:
    return 0
  sayininBasi = sayininOlduguYer
  sayininSonu = 0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return 0
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
  return icerik[sayininBasi:sayininSonu]

def regaVeriBul(icerik):
  veri_rega_no, veri_rega_tarihi = regaNo(satir) , ''
  # data_text alanında yer alan içeriği rega_tarihi bulmak için temizleyeceğiz.
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
  icerik = icerik.replace('.','')
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  

  icerik.strip()
  if icerik=='':
    return veri_rega_no, veri_rega_tarihi

  # Eğer resmiGazeteyeregaNoIsle fonksiyonu sayıyı bulamazsa aşağıdaki koşul çalışmayacaktır.
  # Eğer bulursa resmi gazete tarihi resmi gazete sayısından önce olduğu için tüm belgede arama yapmamıza gerek kalmaz
  if veri_rega_no:
    icerik = icerik[:icerik.find(veri_rega_no)]

  # Şimdi rega_tarihi bulacağız
  # Resmi gazete tarihi XX AY XXXX şeklinde bir biçime sahip yani AY yazı ile veriliyor!
  # Resmi gazete tarihi kuruluş tarihinden sonra veriliyor

  # Öyleyse aramayı baştan da daraltabiliriz.
  kurulusTarihininOlduguYer = icerik.find('tarihi')
  if kurulusTarihininOlduguYer!=-1:
    icerik = icerik[kurulusTarihininOlduguYer+15:]
  #sayı ve tarih birbirine yakın olduğu için baştan değil de sondan arama yapmak için içeriği bir de ters çevirelim.
  icerik = icerik[::-1]

  # Aramaya başlamadan istisnaları da çıkartalım
  # 'tarihlive' diye baslayan içerikten doğrudan çekim yapılabilir.
  tarihAcikISE=icerik.find('evilhirat')
  if tarihAcikISE!=-1:
    istisna = []
    istisna.append(icerik[tarihAcikISE+9:tarihAcikISE+13][::-1])
    istisnaAY_Basla = tarihAcikISE+13
    while not icerik[istisnaAY_Basla].isnumeric():
      istisnaAY_Basla+=1
      if istisnaAY_Basla==len(icerik):
        return veri_rega_no, veri_rega_tarihi
    istisna.append(icerik[tarihAcikISE+13:istisnaAY_Basla][::-1])
    ay = str(ayinAgirliginaBakipTahminYap(istisna[1]))
    if len(ay)==1:
      ay = '0' + ay
    istisna[1] = ay
    if istisnaAY_Basla+2<len(icerik):
      if icerik[istisnaAY_Basla+1].isnumeric():
        istisna.append(icerik[istisnaAY_Basla:istisnaAY_Basla+2])
      else:
        istisna.append(icerik[istisnaAY_Basla:istisnaAY_Basla+1])
    else:
        istisna.append(icerik[istisnaAY_Basla-2:istisnaAY_Basla])
    veri_rega_tarihi = str(str(istisna[0])+'-'+str(istisna[1])+'-'+str(istisna[2])).replace("'",'').replace('[','').replace(']','').replace(',','').replace(' ','')
    return veri_rega_no, veri_rega_tarihi

  # Şimdi aramaya başlayabiliriz.
  bulunan = []
  # İlk bulduğumuz sayı tarihin başladığı ilk bulduğumuz metin ay ve ilk bulduğumuz sayı tekrar tarihin başladığı yer olabilir.
  sayininBasi = 0
  sayininSonu = 0
  if len(icerik)<2:
    return veri_rega_no, veri_rega_tarihi
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return veri_rega_no, veri_rega_tarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return veri_rega_no, veri_rega_tarihi
  bulunan.append(icerik[sayininBasi:sayininSonu])
  # Dokümandaki ilk sayıyı aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return veri_rega_no, veri_rega_tarihi
  bulunan.append(icerik[sayininBasi:sayininSonu])
  # Dokümandaki ay olarak dusunduğumuz kısmı aldık
  sayininBasi = sayininSonu
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return veri_rega_no, veri_rega_tarihi
  bulunan.append(icerik[sayininBasi:sayininSonu])
  # Dokümandaki yıl olarak dusunduğumuz sayıyı aldık!
  ay = str(ayinAgirliginaBakipTahminYap(bulunan[1]))
  if len(ay)==1:
    ay = '0' + ay
  bulunan[1] = ay
  if len(bulunan[2])==1:
    bulunan[2] = str(bulunan[2]) + '0'
  bulunan[2] = bulunan[2][::-1]
  bulunan[0] = bulunan[0][::-1]
  veri_rega_tarihi = str(str(bulunan[2])+'-'+str(bulunan[1])+'-'+str(bulunan[0])).replace("'",'').replace('[','').replace(']','').replace(',','').replace(' ','')
  return veri_rega_no, veri_rega_tarihi

In [7]:
def mukerrer_noBul(icerik):
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
#  icerik = icerik.replace('.','') Mükerrer için önemli
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  
  mukerrerinYeri = icerik.find('mukerrer')
  if mukerrerinYeri==-1:
    return 0.0
  elif icerik[mukerrerinYeri-1]=='.' and icerik[mukerrerinYeri-2].isnumeric():
    if icerik[mukerrerinYeri-2].isnumeric():
      return float(int(icerik[mukerrerinYeri-2]))
    else:
      return 1.0
  elif icerik[mukerrerinYeri-1]!='.' and icerik[mukerrerinYeri-1]!='(':
    return float(1)
  elif len(icerik)>mukerrerinYeri+9:
    return 0.0
  elif mukerrerinYeri+9<len(icerik) and icerik[mukerrerinYeri-1]=='(' and icerik[mukerrerinYeri+9]==')':
    return 1.0
  else:
    return float(0)

In [8]:
def rega_mevzuatVeriBul(icerik):
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
#  icerik = icerik.replace('.','') veri çekmek için önemli
#  icerik = icerik.replace(':','') veri çekmek için önemli
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  

  rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi = '', '', '', ''

# Şimdi aramaya başlayabiliriz.
  rega_tar = []
  # Önce Resmi Gazete Tarihini Çekeceğiz.
  sayininBasi = 0
  sayininSonu = 0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin YILINI aldık!
  
  # Şimdi Resmi Gazete Sayısını Çekeceğiz.
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_sayisi = icerik[sayininBasi:sayininSonu]
  # Resmi Gazetenin SAYISINI aldık!

  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_sayisi = icerik[sayininBasi:sayininSonu]
  # Kanunun / Mevzuatın SAYISINI aldık!
  mevzuat_tar = []
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Kanunun / Mevzuatın GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Kanunun / Mevzuatın AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Kanunun / Mevzuatın YILINI aldık!

  # 'rega_tarihi' YIL-AY-GÜN, 'rega_no', 'mevzuat_no', 'mevzuat_tarihi' YIL-AY-GUN
  rega_t = ""
  rega_t = str(rega_tar[2]) + '-'
  if len(str(rega_tar[1]))==2:
    rega_t+=str(rega_tar[1])
  else:
    rega_t+='0'+str(rega_tar[1])
  rega_t += '-'
  if len(str(rega_tar[0]))==2:
    rega_t += str(rega_tar[0])
  else:
    rega_t +=str('0'+str(rega_tar[0]))
  
  mevzuatTarihi = ""
  mevzuatTarihi += str(mevzuat_tar[2])[:4] + '-'
  if len(str(mevzuat_tar[1]))==2:
    mevzuatTarihi += str(mevzuat_tar[1])
  else:
    mevzuatTarihi +='0'+ str(mevzuat_tar[1])
  mevzuatTarihi +='-'
  if len(str(mevzuat_tar[0]))==2:
    mevzuatTarihi += str(mevzuat_tar[0])
  else:
    mevzuatTarihi +='0'+ str(mevzuat_tar[0])

  return rega_t, rega_sayisi[:5] if len(rega_sayisi)<=6 else rega_sayisi[:-4], mevzuat_sayisi, mevzuatTarihi

In [16]:
def donem_sira_noBul(icerik):
  icerik = str(icerik).lower()
  icerik = icerik.replace('(','')
  icerik = icerik.replace(')','')
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
  icerik = icerik.replace('.','')
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','c')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')
  icerik = icerik.replace('@','')
  icerik = icerik.replace('|','1')
  

  yasamaYillari = [[1920,1923],[1923,1927],[1927,1931],[1931,1934],[1935,1939],[1939,1943],[1943,1946],[1946,1950],[1950,1954],[1954,1957],[1957,1960],[1961,1965],[1965,1965],[1969,1973],[1973,1977],[1977,1980],[1983,1987],[1987,1991],[1991,1995],[1996,1999],[1999,2002],[2002,2007],[2007,2011],[2011,2015],[2015,2015],[2015,2018],[2018,2022]]

  # Şimdi aramaya başlayabiliriz.
  # Önce DÖNEMİ Çekeceğiz.
  donem, sira = 0, 0
  sayininBasi = icerik.find('donem')
  if sayininBasi!=-1:
    sayininSonu = 0
    while not icerik[sayininBasi].isnumeric():
      sayininBasi+=1
      if sayininBasi==len(icerik):
        return donem, sira
    sayininSonu = sayininBasi
    while icerik[sayininSonu].isnumeric():
      sayininSonu+=1
      if sayininSonu==len(icerik):
        return donem, sira
    donem = icerik[sayininBasi:sayininSonu]
  if donem==0:
    sayininBasi = icerik.find('devre')
    if sayininBasi != -1:
      if 'devreiii' in icerik:
        donem=3
      elif 'devreii' in icerik:
        donem=2
      elif 'devreiv' in icerik:
        donem=4
      elif 'devreix' in icerik:
        donem=9
      elif 'devrei' in icerik:
        donem=1
      elif 'devreviii' in icerik:
        donem=8
      elif 'devrevii' in icerik:
        donem=7
      elif 'devrevi' in icerik:
        donem=6
      elif 'devrev' in icerik:
        donem=5
      elif 'devrexiv' in icerik:
        donem=14
      elif 'devrexiii' in icerik:
        donem=13
      elif 'devrexix' in icerik:
        donem=19
      elif 'devrexii' in icerik:
        donem=12
      elif 'devrexi' in icerik and not ('devrexict' in icerik):
        donem=11
      elif 'devrexviii' in icerik:
        donem=18
      elif 'devrexvii' in icerik:
        donem=17
      elif 'devrexvi' in icerik:
        donem=16
      elif 'devrexv' in icerik:
        donem=15
      elif 'devrexx' in icerik:
        donem=20
      elif 'devrex' in icerik:
        donem=10
    elif icerik.find('milletmecli')!=-1 and icerik.find('buyukmilletmecli')==-1:
      donem = 1
    elif icerik.find('tarihli')!=-1:
      yil = icerik[icerik.find('tarihli')-4:icerik.find('tarihli')]
      yil = yil.replace('i','1')
      yil = yil.replace('h','1')
      yil_temiz = ''
      for k in yil:
        if k in '0123456789':
          yil_temiz += k
        else:
          continue
      if yil_temiz=='':
        yil=0
      else:
        yil = int(yil_temiz)
      degisim = 0
      for aralik in yasamaYillari:
        if aralik[0]<yil and aralik[1]>=yil:
          donem = (yasamaYillari.index(aralik)+1)
          degisim = 1
      if not(degisim):
        donem = 1
    else:
      sayininBasi = 0
      sayininSonu = 0
      while not icerik[sayininBasi].isnumeric():
        sayininBasi+=1
        if sayininBasi==len(icerik):
          return donem, sira
      sayininSonu = sayininBasi
      while icerik[sayininSonu].isnumeric():
        sayininSonu+=1
        if sayininSonu==len(icerik):
          return donem, sira
      donem = icerik[sayininBasi:sayininSonu]
  # Şimdi sırada sıra var!
  sayininBasi = icerik.find('sayi')
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return donem, sira
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return donem, sira
  sira = icerik[sayininBasi:sayininSonu]
  if len(sira)>4:
    sira=sira[:4]
  kesit = icerik[sayininSonu:sayininSonu+8]
  if 'ci' in kesit or 'cu' in kesit:
    ekno = ''
    for karakter in kesit:
      if karakter in '0123456789':
        ekno+=karakter
    if ekno=='':
      ekno='1'
    sira += ' ek ' + ekno
  elif 'ek' in kesit:
    sira += ' ek 1'
  if len(str(donem))>2:
    donem=str(donem)[:2]
  return str(donem), sira

In [10]:
def genelgeVeriBul(icerik):
  icerik = str(icerik).lower()
  icerik = icerik.replace('(','')
  icerik = icerik.replace(')','')
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
  icerik = icerik.replace('.','')
  icerik = icerik.replace(':','')
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','c')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')
  icerik = icerik.replace('@','')
  icerik = icerik.replace('|','1')
  icerik = icerik.replace('2ÖÎ','201')
  icerik = icerik.replace('2Ö1','201')
  
  # Şimdi aramaya başlayabiliriz.
  mevzuatTarihi, durum, no = '', '', ''
  
  # Önce TARİH'i bulalım!
  mevzuat_tar = []
  sayininBasi, sayininSonu = icerik.find('tarih'), 0
  if sayininBasi==-1:
    sayininBasi=0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return mevzuatTarihi, 'durum1', no
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return mevzuatTarihi, 'durum1', no
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Mevzuatın GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return mevzuatTarihi, 'durum2', no
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return mevzuatTarihi, 'durum2', no
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Mevzuatın AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return mevzuatTarihi, 'durum3', no
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return mevzuatTarihi, 'durum3', no
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Mevzuatın YILINI aldık!

  mevzuatTarihi = ""
  mevzuatTarihi += str(mevzuat_tar[2])[:4] + '-'
  if len(str(mevzuat_tar[1]))==2:
    mevzuatTarihi += str(mevzuat_tar[1])
  else:
    mevzuatTarihi +='0'+ str(mevzuat_tar[1])
  mevzuatTarihi +='-'
  if len(str(mevzuat_tar[0]))==2:
    mevzuatTarihi += str(mevzuat_tar[0])
  else:
    mevzuatTarihi +='0'+ str(mevzuat_tar[0])

  # Şimdi NO Çekeceğiz.
  no, mevzuatnoVARMI, tarih = '', 1, ''
  sayininBasi = icerik.find('genel')
  if sayininBasi==-1:
    sayininBasi = 0
  else:
    sayininSonu = 0
    while not icerik[sayininBasi].isnumeric():
      sayininBasi+=1
      if sayininBasi==len(icerik):
        return mevzuatTarihi, 'durum4', no
    sayininSonu = icerik.find('tarih')
    no = icerik[sayininBasi:sayininSonu]
  if no!='':
    mevzuatnoVARMI=0

  # 'mevzuat_tarihi' YIL-AY-GÜN, 'mevzuat_no' var mi, no

  return mevzuatTarihi, '0', str(no)

In [11]:
from numpy import nan
def CBK_VeriBul(icerik,satir):
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
#  icerik = icerik.replace('.','') veri çekmek için önemli
#  icerik = icerik.replace(':','') veri çekmek için önemli
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  

  rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi = '','','',''
  # Şimdi aramaya başlayabiliriz.
  rega_tar = []
  # Önce Resmi Gazete Tarihini Çekeceğiz.
  sayininBasi = 0
  sayininSonu = 0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin YILINI aldık!
  
  # Şimdi Resmi Gazete Sayısını Çekeceğiz.
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
  rega_sayisi = icerik[sayininBasi:sayininSonu]
  # Resmi Gazetenin SAYISINI aldık!

  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
  mevzuat_sayisi = icerik[sayininBasi:sayininSonu]
  # Kanunun / Mevzuatın SAYISINI aldık!
  mevzuat_tar = []
  noktaninYeri = veriIskeleti['data_text'][satir][::-1].find('.')
  if noktaninYeri < 7:
    mevzuatTarihi = nan
  else:
    icerik = veriIskeleti['data_text'][satir][::-1][:15]
    # Şimdi aramaya başlayabiliriz.
    bulunan = []
    # İlk bulduğumuz sayı tarihin başladığı ilk bulduğumuz metin ay ve ilk bulduğumuz sayı tekrar tarihin başladığı yer olabilir.
    sayininBasi = 0
    sayininSonu = 0
    while not icerik[sayininBasi].isnumeric():
      sayininBasi+=1
      if sayininBasi==len(icerik):
        return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
    sayininSonu = sayininBasi
    while icerik[sayininSonu].isnumeric():
      sayininSonu+=1
    bulunan.append(icerik[sayininBasi:sayininSonu])
    # Dokümandaki ilk sayıyı aldık!
    sayininBasi = sayininSonu
    while not icerik[sayininSonu].isnumeric():
      sayininSonu+=1
      if sayininSonu==len(icerik):
        return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
    bulunan.append(icerik[sayininBasi:sayininSonu])
    # Dokümandaki ay olarak dusunduğumuz kısmı aldık
    sayininBasi = sayininSonu
    while icerik[sayininSonu].isnumeric():
      sayininSonu+=1
      if sayininSonu==len(icerik):
        return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
    bulunan.append(icerik[sayininBasi:sayininSonu])
    # Dokümandaki yıl olarak dusunduğumuz sayıyı aldık!
    ay = str(ayinAgirliginaBakipTahminYap(bulunan[1]))
    if len(ay)==1:
      ay = '0' + ay
    bulunan[1] = ay
    if len(bulunan[2])==1:
      bulunan[2] = str(bulunan[2]) + '0'
    bulunan[2] = bulunan[2][::-1]
    bulunan[0] = bulunan[0][::-1]
    mevzuatTarihi = str(str(bulunan[0])+'-'+str(bulunan[1])+'-'+str(bulunan[2])).replace("'",'').replace('[','').replace(']','').replace(',','').replace(' ','')

  # 'rega_tarihi' YIL-AY-GÜN, 'rega_no', 'mevzuat_no', 'mevzuat_tarihi' YIL-AY-GUN
  rega_t = ""
  rega_t = str(rega_tar[2]) + '-'
  if len(str(rega_tar[1]))==2:
    rega_t+=str(rega_tar[1])
  else:
    rega_t+='0'+str(rega_tar[1])
  rega_t += '-'
  if len(str(rega_tar[0]))==2:
    rega_t += str(rega_tar[0])
  else:
    rega_t +=str('0'+str(rega_tar[0]))

  return rega_t, rega_sayisi[:5], mevzuat_sayisi, mevzuatTarihi

In [12]:
def tuzukVeriBul(icerik):
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
#  icerik = icerik.replace('.','') veri çekmek için önemli
#  icerik = icerik.replace(':','') veri çekmek için önemli
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  

  rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi = '', '', '', ''

  # Şimdi aramaya başlayabiliriz.
  rega_tar = []
  # Önce Resmi Gazete Tarihini Çekeceğiz.
  sayininBasi = 0
  sayininSonu = 0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin YILINI aldık!
  
  # Şimdi Resmi Gazete Sayısını Çekeceğiz.
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  rega_sayisi = icerik[sayininBasi:sayininSonu]
  # Resmi Gazetenin SAYISINI aldık!

  mevzuat_tar = []
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Kanunun / Mevzuatın GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Kanunun / Mevzuatın AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])
  # Kanunun / Mevzuatın YILINI aldık!

  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_sayisi = icerik[sayininBasi:sayininSonu]
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi, mevzuat_sayisi, mevzuatTarihi
  mevzuat_sayisi += '/' + icerik[sayininBasi:sayininSonu]

  # Kanunun / Mevzuatın SAYISINI aldık!

  # 'rega_tarihi' YIL-AY-GÜN, 'rega_no', 'mevzuat_no', 'mevzuat_tarihi' YIL-AY-GUN
  rega_t = ""
  rega_t = str(rega_tar[2]) + '-'
  if len(str(rega_tar[1]))==2:
    rega_t+=str(rega_tar[1])
  else:
    rega_t+='0'+str(rega_tar[1])
  rega_t += '-'
  if len(str(rega_tar[0]))==2:
    rega_t += str(rega_tar[0])
  else:
    rega_t +=str('0'+str(rega_tar[0]))
  
  mevzuatTarihi = ""
  mevzuatTarihi += str(mevzuat_tar[2])[:4] + '-'
  if len(str(mevzuat_tar[1]))==2:
    mevzuatTarihi += str(mevzuat_tar[1])
  else:
    mevzuatTarihi +='0'+ str(mevzuat_tar[1])
  mevzuatTarihi +='-'
  if len(str(mevzuat_tar[0]))==2:
    mevzuatTarihi += str(mevzuat_tar[0])
  else:
    mevzuatTarihi +='0'+ str(mevzuat_tar[0])

  return rega_t, rega_sayisi[:5], mevzuat_sayisi[:-4] if len(mevzuat_sayisi)>6 else mevzuat_sayisi, mevzuatTarihi

In [19]:
def yonetmelikVeriBul(icerik,satir):
  gelenIcerik = icerik
  icerik = str(veriIskeleti['data_text'][satir])
# icerik = icerik.replace(' ','') kurum verisi çekmek için önemli!
  icerik = icerik.replace('-','')
#  icerik = icerik.replace(',','') kurum verisi çekmek için önemli
#  icerik = icerik.replace('.','') veri çekmek için önemli
#  icerik = icerik.replace(':','') veri çekmek için önemli
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  

  # Önce kurumu bulacağız!
  icerik = icerik[::-1]
  icerik = icerik[:icerik.find('eddaM')]
  icerik = icerik[::-1]
  icerik = icerik[icerik.find('hükümlerini')+11:]
  if icerik[0]==' ':
    icerik = icerik[1:]
  icerik = icerik[:icerik.find(' yürütür.')]
  icerik = icerik.replace('/','')
  icerik = icerik.replace('(','')
  icerik = icerik.replace(')','')
  temiz = ''
  for karakter in icerik:
    if karakter in '0123456789':
      continue
    else:
      temiz += karakter
  icerik = temiz
  icerik = icerik.replace('Cumhurbaşkanı','Cumhurbaşkanlığı')
  icerik = icerik.replace('Bakanı','Bakanlığı')
  icerik = icerik.replace('Kurumu Başkanı','Kurumu')
  icerik = icerik.replace('Müdürü','Müdürlüğü')
  if icerik.find('Rektörü')!=-1:
    icerik='Üniversite'
  if icerik.find('olduğu Bakan')!=-1:
    icerik='Başbakanlık'
  icerik = icerik.replace('sayilicetvel','')
  icerik = icerik.replace('Değişik ibare:RG ','')
  kurum = icerik

  # Artık kurumu bulduğumuza göre gelen iceriği (gelenIcerik) temizleyip devam edeceğiz
  icerik = gelenIcerik
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
#  icerik = icerik.replace('.','') veri çekmek için önemli
#  icerik = icerik.replace(':','') veri çekmek için önemli
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')  

  rega_t, rega_sayisi = '',''
  # Şimdi aramaya başlayabiliriz.
  rega_tar = []
  # Önce Resmi Gazete Tarihini Çekeceğiz.
  sayininBasi = 0
  sayininSonu = 0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi
  rega_tar.append(icerik[sayininBasi:sayininSonu])
  # Resmi Gazetenin YILINI aldık!
  
  # Şimdi Resmi Gazete Sayısını Çekeceğiz.
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi
  rega_sayisi = icerik[sayininBasi:sayininSonu]
  # Resmi Gazetenin SAYISINI aldık!

  mevzuat_tar = []
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return rega_t, rega_sayisi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return rega_t, rega_sayisi
  mevzuat_tar.append(icerik[sayininBasi:sayininSonu])

  # 'rega_tarihi' YIL-AY-GÜN, 'rega_no', 'kurum'
  rega_t = ""
  rega_t = str(rega_tar[2]) + '-'
  if len(str(rega_tar[1]))==2:
    rega_t+=str(rega_tar[1])
  else:
    rega_t+='0'+str(rega_tar[1])
  rega_t += '-'
  if len(str(rega_tar[0]))==2:
    rega_t += str(rega_tar[0])
  else:
    rega_t +=str('0'+str(rega_tar[0]))

  return rega_t, rega_sayisi[:5]

In [14]:
def ozelgeVeriBul(satir):
  tarihinYeri = str(veriIskeleti['data_text'][satir]).find('tarih')
  if tarihinYeri==-1:
    tarihinYeri = len(str(veriIskeleti['data_text'][satir]))
  konununYeri = str(veriIskeleti['data_text'][satir]).find('konu')
  if tarihinYeri>konununYeri:
    icerik=str(veriIskeleti['data_text'][satir])[:konununYeri]
    icerik = icerik[::-1]
  else:
    icerik = str(veriIskeleti['data_text'][satir])[tarihinYeri:tarihinYeri+20]
  icerik = str(icerik).lower()
  icerik = icerik.replace(' ','')
  icerik = icerik.replace('-','')
  icerik = icerik.replace(',','')
#  icerik = icerik.replace('.','') veri çekmek için önemli
#  icerik = icerik.replace(':','') veri çekmek için önemli
  icerik = icerik.replace(';','')
  icerik = icerik.replace('\\','')
  icerik = icerik.replace('\n','')
  icerik = icerik.replace('\r','')
  icerik = icerik.replace('_','')
  icerik = icerik.replace('─','')
  icerik = icerik.replace('â','a')
  icerik = icerik.replace('å','a')
  icerik = icerik.replace('æ','')
  icerik = icerik.replace('ç','c')
  icerik = icerik.replace('ğ','g')
  icerik = icerik.replace('ı','i')
  icerik = icerik.replace('i̇','i')
  icerik = icerik.replace('î','i')
  icerik = icerik.replace('ṁ','m')
  icerik = icerik.replace('ṅ','n')
  icerik = icerik.replace('ö','o')
  icerik = icerik.replace('ṙ','r')
  icerik = icerik.replace('ṡ','s')
  icerik = icerik.replace('ş','s')
  icerik = icerik.replace('ü','u')
  icerik = icerik.replace('©','')
  icerik = icerik.replace('&','')
  icerik = icerik.replace('ý','ı')
  icerik = icerik.replace('…','')
  mevzuat_tarihi = ''
  # Şimdi aramaya başlayabiliriz.
  mev_tar = []
  # Önce Tarihini Çekeceğiz.
  sayininBasi = 0
  sayininSonu = 0
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return mevzuat_tarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return mevzuat_tarihi
  mev_tar.append(icerik[sayininBasi:sayininSonu])
  # GÜNÜNÜ aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return mevzuat_tarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return mevzuat_tarihi
  mev_tar.append(icerik[sayininBasi:sayininSonu])
  # AYINI aldık!
  sayininBasi = sayininSonu
  while not icerik[sayininBasi].isnumeric():
    sayininBasi+=1
    if sayininBasi==len(icerik):
      return mevzuat_tarihi
  sayininSonu = sayininBasi
  while icerik[sayininSonu].isnumeric():
    sayininSonu+=1
    if sayininSonu==len(icerik):
      return mevzuat_tarihi
  mev_tar.append(icerik[sayininBasi:sayininSonu])
  # YILINI aldık!

  if tarihinYeri>konununYeri:
    # 'mevzuat_tarihi' YIL-AY-GÜN
    t = ""
    t = str(mev_tar[0][::-1]) + '-'
    if len(str(mev_tar[1]))==2:
      t+=str(mev_tar[1][::-1])
    else:
      t+='0'+str(mev_tar[1][::-1])
    t += '-'
    if len(str(mev_tar[2]))==2:
      t += str(mev_tar[2][::-1][-2:])
    else:
      t +=str(str(mev_tar[2][::-1][-2:]))
  else:
    # 'mevzuat_tarihi' YIL-AY-GÜN
    t = ""
    t = str(mev_tar[2]) + '-'
    if len(str(mev_tar[1]))==2:
      t+=str(mev_tar[1])
    else:
      t+='0'+str(mev_tar[1])
    t += '-'
    if len(str(mev_tar[0]))==2:
      t += str(mev_tar[0])
    else:
      t +=str(str(mev_tar[0]))
  return str(t)

In [17]:
def veriDoldur(satir):
  icerik = veriIskeleti['data_text'][satir]
  # Verimizi daha önce izah ettiğimiz gibi iki parcaya ayırıp ilk parçayı alıyoruz!
  b1, b2 = icerik.find('sayı'), icerik.find('madde1')
  b1 = max(b1,200)
  bolmeYeri = b2 if b2!=-1 else b1 if b1!=-1 else 300
  arastirilacakParca = veriIskeleti['data_text'][satir][:bolmeYeri]
  dokumaninKategorisi = kategoriBul(arastirilacakParca)
  veriIskeleti.loc[satir, 'kategori'] = dokumaninKategorisi
  if dokumaninKategorisi == 'Kanun':
#    rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi ve madde_sayisi
    veriIskeleti.loc[satir, 'rega_tarihi'], veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'mevzuat_no'], veriIskeleti.loc[satir, 'mevzuat_tarihi'] = rega_mevzuatVeriBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'mukerrer_no'] = mukerrer_noBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'madde_sayisi'] = madde_sayisiBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Kanun Hükmünde Kararname':
#    rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi ve madde_sayisi
    veriIskeleti.loc[satir, 'rega_tarihi'], veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'mevzuat_no'], veriIskeleti.loc[satir, 'mevzuat_tarihi'] = rega_mevzuatVeriBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'mukerrer_no'] = mukerrer_noBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'madde_sayisi'] = madde_sayisiBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Resmi Gazete':
#    rega_no, mukerrer_no, rega_tarihi
    veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'rega_tarihi'] = regaVeriBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Komisyon Raporu':
#    donem ve sira_no
    foksiyondanDonenVeriler = donem_sira_noBul(arastirilacakParca)
    if foksiyondanDonenVeriler[0] in ['21','22','23','24','25','26','27']:
      veriIskeleti.loc[satir, 'donem'] = foksiyondanDonenVeriler[0] + ' Dönem'
    veriIskeleti.loc[satir, 'sira_no'] = foksiyondanDonenVeriler[1]

  elif dokumaninKategorisi == 'Genelge':
#    mevzuat_tarihi ve varsa mevzuat_no yoksa belge_sayi
    donut = genelgeVeriBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'mevzuat_tarihi'] = donut[0]
    if donut[1]!=0:
      veriIskeleti.loc[satir, 'mevzuat_no'] = donut[2]
    else:
      veriIskeleti.loc[satir, 'belge_sayisi'] = donut[2]

  elif dokumaninKategorisi == 'Cumhurbaşkanlığı Kararnamesi':
#    rega_no, mukerrer_no, rega_tarihi, mevzuat_no, madde_sayisi ve varsa mevzuat_tarihi
    veriIskeleti.loc[satir, 'rega_tarihi'], veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'mevzuat_no'], veriIskeleti.loc[satir, 'mevzuat_tarihi'] = CBK_VeriBul(arastirilacakParca,satir)
    veriIskeleti.loc[satir, 'mukerrer_no'] = mukerrer_noBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'madde_sayisi'] = madde_sayisiBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Tüzük':
#    rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi
    veriIskeleti.loc[satir, 'rega_tarihi'], veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'mevzuat_no'], veriIskeleti.loc[satir, 'mevzuat_tarihi'] = tuzukVeriBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'mukerrer_no'] = mukerrer_noBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'madde_sayisi'] = madde_sayisiBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Yönetmelik':
#    rega_no, mukerrer_no, rega_tarihi, kurum, madde_sayisi
    veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'rega_tarihi'] = yonetmelikVeriBul(arastirilacakParca,satir)
    veriIskeleti.loc[satir, 'mukerrer_no'] = mukerrer_noBul(arastirilacakParca)
    veriIskeleti.loc[satir, 'madde_sayisi'] = madde_sayisiBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Tebliğ':  # Yönetmelikte fonksiyonu tebliğ için de verileri döndürebiliyor!
#    rega_no, mukerrer_no, rega_tarihi, kurum
    veriIskeleti.loc[satir, 'rega_no'], veriIskeleti.loc[satir, 'rega_tarihi'] = yonetmelikVeriBul(arastirilacakParca,satir)
    veriIskeleti.loc[satir, 'mukerrer_no'] = mukerrer_noBul(arastirilacakParca)

  elif dokumaninKategorisi == 'Özelge':
#    mevzuat_tarihi
    veriIskeleti.loc[satir, 'mevzuat_tarihi'] = ozelgeVeriBul(satir)

In [20]:
satirToplami, sutunToplami = veriIskeleti.shape
for satir in range(satirToplami):
  veriDoldur(satir)
print('İlgili tüm kayıtlar doldurulmuştur.')

İlgili tüm kayıtlar doldurulmuştur.


In [24]:
veriIskeleti.drop(['data_text', 'url','kanunum_url'], axis=1, inplace=True)

veriIskeleti.to_csv('ifl_garaji_10100_TDDI_Kamuda_Mevzuat_Arama_Motoru_Gelistirme_Yarismasi.csv',index=False)
veriIskeleti.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4126 entries, 0 to 4125
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4126 non-null   int64  
 1   kategori        4126 non-null   object 
 2   baslik          0 non-null      float64
 3   rega_no         2514 non-null   object 
 4   mukerrer_no     4126 non-null   int64  
 5   rega_tarihi     2514 non-null   object 
 6   kurum           0 non-null      float64
 7   mevzuat_no      1108 non-null   object 
 8   belge_sayi      0 non-null      float64
 9   mevzuat_tarihi  1837 non-null   object 
 10  donem           298 non-null    object 
 11  sira_no         504 non-null    object 
 12  madde_sayisi    4126 non-null   int64  
dtypes: float64(3), int64(3), object(7)
memory usage: 419.2+ KB
